In [1]:
!nvidia-smi

Sun Nov  8 19:54:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install keras-tcn

     |████████████████████████████████| 378kB 5.8MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import os
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal, spatial, stats
from fastdtw import fastdtw
from scipy.signal import find_peaks
import pywt
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, Dense, Conv1D, Conv2D, MaxPooling2D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy, Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import pickle
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
from tcn import TCN, tcn_full_summary

In [5]:
os.chdir('gdrive/My Drive/Weave/Earables Project/Final Experiments/')

In [6]:
data = pickle.load(open('data/processed_df.pkl', 'rb'))

In [7]:
labels = data.iloc[:,-3:-2]
ohe = OneHotEncoder()
ohe.fit(labels)

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [12]:
subjects = list(np.unique(data.subject))
# print(subjects)
test_i = 7
test_sub = subjects[test_i]

left_cols = [c for c in data.columns if 'Left' in c]
sub_data = data[data.subject == test_sub]
sub_data = sub_data.drop(columns = left_cols)
idx = list(sub_data.index)
indices = np.arange(idx[0][0], idx[-1][0]+1)
np.random.shuffle(indices)

k=5
folds = []
fold_len = len(indices)//k
for i in range(k):
    st = i * fold_len
    en = (i+1) * fold_len 
    fold_i = indices[st:en]
    fold_d = sub_data.loc[fold_i]
    folds.append(fold_d)

n_classes = 32
print(sub_data.shape)

(10800, 9)


In [15]:
for test_i in range(0,7):
    test_sub = subjects[test_i]
    print('Test subject:', test_sub)
    left_cols = [c for c in data.columns if 'Left' in c]
    sub_data = data[data.subject == test_sub]
    sub_data = sub_data.drop(columns = left_cols)
    idx = list(sub_data.index)
    indices = np.arange(idx[0][0], idx[-1][0]+1)
    np.random.shuffle(indices)

    k=5
    folds = []
    fold_len = len(indices)//k
    for i in range(k):
        st = i * fold_len
        en = (i+1) * fold_len 
        fold_i = indices[st:en]
        fold_d = sub_data.loc[fold_i]
        folds.append(fold_d)

    n_classes = 32
    print(sub_data.shape)
    for test_fold in range(3):
        test_data = folds[test_fold]
        train_data = []
        for fold_i, fold_d in enumerate(folds):
            if fold_i != test_fold:
                train_data.append(fold_d)

        train_data = pd.concat(train_data)

        X_train = train_data.iloc[:,:-3].to_numpy()
        y_train = train_data.iloc[:,-3:-2]
        X_test = test_data.iloc[:,:-3].to_numpy()
        y_test = test_data.iloc[:,-3:-2]
        y_train = ohe.transform(y_train).toarray()
        y_test = ohe.transform(y_test).toarray()

        #--------------------------------------------
        tmp = []
        for i in range(len(X_train)):
            ch = []
            for j in range(6):
                ch.append(X_train[i][j])
            ch = np.array(ch).T
            tmp.append(ch)
        X_train = np.array(tmp)

        tmp = []
        for i in range(len(X_test)):
            ch = []
            for j in range(6):
                ch.append(X_test[i][j])
            ch = np.array(ch).T
            tmp.append(ch)
        X_test = np.array(tmp)
        del tmp, ch
        # print(X_train.shape, y_train.shape)
        # print(X_test.shape, y_test.shape)

        # --------------------------------------
        batch_size = 32
        input_shape = (100, 6)

        img_input = Input(shape=input_shape)
        x = TCN(return_sequences=False, nb_filters=64)(img_input)
        # x = TCN(return_sequences=False, nb_filters=128)(x)
        x = Dense(n_classes, activation='sigmoid', name='output')(x)

        model = Model(img_input, x, name='tcn_right_sub_{}_fold_{}'.format(test_sub, test_fold+1))
        # model.summary()

        # -----------------------------------------------
        model_name = 'tcn_right_sub_{}_fold_{}'.format(test_sub, test_fold+1)
        optimizer = Adam(learning_rate=0.001)
        # loss = BinaryCrossentropy()
        # model.load_weights('checkpoint/{}.h5'.format(model_name))
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics='accuracy')

        checkpoint = ModelCheckpoint('checkpoint/left_vs_right/{}.h5'.format(model_name), monitor='val_loss', verbose=0, mode='min',save_best_only=True)
        earlystopping = EarlyStopping(monitor='val_loss', patience=20, verbose=0)
        reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, min_lr=0.000001)

        # ----------------------------------------------
        print('started training...')
        history = model.fit(X_train, y_train,
                            validation_data=(X_test, y_test),
                            batch_size=batch_size,
                            epochs=100,
                            # workers=4,
                            callbacks=[checkpoint, earlystopping],
                            shuffle=True,
                            verbose=0)

        # -----------------------------------------------
        labels = ohe.categories_[0]
        model.load_weights('checkpoint/left_vs_right/{}.h5'.format(model_name))
        print('Model loaded: checkpoint/left_vs_right/{}.h5'.format(model_name))
        pred = model.predict(X_test)
        y_pred = [labels[i] for i in np.argmax(pred, axis=1)]
        y_true = [labels[i] for i in np.argmax(y_test, axis=1)]
        print(accuracy_score(y_true, y_pred))

Test subject: 11
(10850, 9)
started training...
Model loaded: checkpoint/left_vs_right/tcn_right_sub_11_fold_1.h5
0.7990783410138249
started training...
Model loaded: checkpoint/left_vs_right/tcn_right_sub_11_fold_2.h5
0.7958525345622119
started training...
Model loaded: checkpoint/left_vs_right/tcn_right_sub_11_fold_3.h5
0.7774193548387097
Test subject: 12
(10849, 9)
started training...
Model loaded: checkpoint/left_vs_right/tcn_right_sub_12_fold_1.h5
0.7036866359447005
started training...
Model loaded: checkpoint/left_vs_right/tcn_right_sub_12_fold_2.h5
0.7511520737327189
started training...
Model loaded: checkpoint/left_vs_right/tcn_right_sub_12_fold_3.h5
0.7202764976958526
Test subject: 13
(10200, 9)
started training...
Model loaded: checkpoint/left_vs_right/tcn_right_sub_13_fold_1.h5
0.9470588235294117
started training...
Model loaded: checkpoint/left_vs_right/tcn_right_sub_13_fold_2.h5
0.9691176470588235
started training...
Model loaded: checkpoint/left_vs_right/tcn_right_sub_13_